# Mediapipe Installation

In [ ]:
!pip install mediapipe


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Video Preprocessing

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from sklearn.model_selection import train_test_split
import zipfile

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

data_path = "/content/drive/MyDrive/newdata"
output_path = "/content/drive/MyDrive/finaloutput"

def alter_speed(frames, factor):
    if factor > 1:
        return frames[::int(factor)]
    elif factor < 1:
        return [frame for frame in frames for _ in range(int(1 / factor))]
    return frames

def crop_frame(frame):
    h, w = frame.shape[:2]
    return frame[int(h * 0.1):int(h * 0.9), int(w * 0.1):int(w * 0.9)]

def add_noise(frame):
    noise = np.random.normal(0, 25, frame.shape).astype(np.uint8)
    return cv2.add(frame, noise)

def apply_blur(frame, kernel_size=5):
    return cv2.GaussianBlur(frame, (kernel_size, kernel_size), 0)

def extract_keypoints_with_augmentation(video_path, speed_factor=1):
    cap = cv2.VideoCapture(video_path)
    keypoints_sequence = []
    frame_sequence = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_sequence.append(frame_rgb)

    cap.release()

    altered_frames = alter_speed(frame_sequence, speed_factor)

    augmented_frames = []
    for frame in altered_frames:
        augmented_frames.append(frame)
        augmented_frames.append(cv2.flip(frame, 1))

    for frame in augmented_frames:
        results = pose.process(frame)
        if results.pose_landmarks:
            keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten()
            keypoints_sequence.append(keypoints)
        else:
            keypoints_sequence.append(np.zeros(99))

    return np.array(keypoints_sequence)

data = []
labels = []
label_map = {}

for idx, person in enumerate(sorted(os.listdir(data_path))):
    person_folder = os.path.join(data_path, person)
    if not os.path.isdir(person_folder):
        continue

    print(f"Processing folder: {person_folder}")
    label_map[idx] = person
    for video_file in os.listdir(person_folder):
        video_path = os.path.join(person_folder, video_file)
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            print(f"Processing video: {video_path}")
            keypoints = extract_keypoints_with_augmentation(video_path, speed_factor=1.5)
            if keypoints.size > 0:
                data.append(keypoints)
                labels.append(idx)


print(f"Number of samples loaded: {len(data)}")
print(f"Number of labels loaded: {len(labels)}")

if len(data) == 0:
    raise ValueError("No data was loaded. Please check your dataset structure and paths.")

data = np.array(data, dtype=object)
labels = np.array(labels)

train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=0.3, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(temp_data, temp_labels, test_size=0.5, random_state=42)

np.savez_compressed(os.path.join(output_path, "processed_data.npz"),
                    train_data=train_data, train_labels=train_labels,
                    val_data=val_data, val_labels=val_labels,
                    test_data=test_data, test_labels=test_labels)

print("Data preprocessing complete!")
print(f"Train Samples: {len(train_data)}, Validation Samples: {len(val_data)}, Test Samples: {len(test_data)}")


Processing folder: /content/drive/MyDrive/newdata/Prarthana
Processing video: /content/drive/MyDrive/newdata/Prarthana/VID20250115105310.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/VID_20250115_102348.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/20250115_105311.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/20250115_105218.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/WhatsApp Video 2025-01-15 at 11.48.53_6b44f21f.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/WhatsApp Video 2025-01-15 at 11.48.58_9541ecde.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/WhatsApp Video 2025-01-15 at 11.49.03_77bf95b8.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/a.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/b.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/c.mp4
Processing video: /content/drive/MyDrive/newdata/Prarthana/Copy of a (1).mp4
Processing video

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/finaloutput/processed_data.npz',allow_pickle=True)
print(data.keys())

KeysView(NpzFile '/content/drive/MyDrive/finaloutput/processed_data.npz' with keys: train_data, train_labels, val_data, val_labels, test_data...)


In [ ]:
data.keys()

KeysView(NpzFile '/content/drive/MyDrive/finaloutput/processed_data.npz' with keys: train_data, train_labels, val_data, val_labels, test_data...)

In [ ]:
train_data=data['train_data']
train_labels = data['train_labels']
val_data = data['val_data']
val_labels = data['val_labels']
test_data = data['test_data']
test_labels = data['test_labels']

print("Train data shape:", train_data.shape)
print("Train label shape:", train_labels.shape)
print("Validation data shape:", val_data.shape)
print("Validation label shape:", val_labels.shape)
print("Test data shape:", test_data.shape)
print("Test label shape:", test_labels.shape)

Train data shape: (71,)
Train label shape: (71,)
Validation data shape: (15,)
Validation label shape: (15,)
Test data shape: (16,)
Test label shape: (16,)


In [ ]:

sequence = train_data[0]
print("Type of first sequence:", type(sequence))
print("Shape of first sequence (if array):", sequence.shape if isinstance(sequence, np.ndarray) else "Not an array")

for i in range(3):
    print(f"Sequence {i} length: {len(train_data[i]) if hasattr(train_data[i], '__len__') else 'Unknown'}")


Type of first sequence: <class 'numpy.ndarray'>
Shape of first sequence (if array): (408, 99)
Sequence 0 length: 408
Sequence 1 length: 548
Sequence 2 length: 452


In [ ]:
for i in range(20):
    print(f"Sequence {i} length: {len(train_data[i]) if hasattr(train_data[i], '__len__') else 'Unknown'}")


Sequence 0 length: 408
Sequence 1 length: 548
Sequence 2 length: 452
Sequence 3 length: 432
Sequence 4 length: 422
Sequence 5 length: 452
Sequence 6 length: 508
Sequence 7 length: 406
Sequence 8 length: 432
Sequence 9 length: 446
Sequence 10 length: 412
Sequence 11 length: 374
Sequence 12 length: 356
Sequence 13 length: 410
Sequence 14 length: 422
Sequence 15 length: 450
Sequence 16 length: 528
Sequence 17 length: 558
Sequence 18 length: 446
Sequence 19 length: 412


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 350
padded_train_data = pad_sequences(train_data, maxlen=max_length, dtype='float32', padding='post', truncating='post')
padded_val_data = pad_sequences(val_data, maxlen=max_length, dtype='float32', padding='post', truncating='post')
padded_test_data = pad_sequences(test_data, maxlen=max_length, dtype='float32', padding='post', truncating='post')

print("Padded train data shape:", padded_train_data.shape)


Padded train data shape: (71, 350, 99)


In [ ]:
print("Padded val data shape:", padded_val_data.shape)
print("Padded test data shape:", padded_test_data.shape)

Padded val data shape: (15, 350, 99)
Padded test data shape: (16, 350, 99)


In [ ]:

padded_train_data = padded_train_data / np.max(padded_train_data)
padded_val_data = padded_val_data / np.max(padded_val_data)
padded_test_data = padded_test_data / np.max(padded_test_data)


In [ ]:

print("Sample labels:", train_labels[:10])
print("Type of labels:", type(train_labels))


Sample labels: [0 3 2 1 2 0 3 2 1 0]
Type of labels: <class 'numpy.ndarray'>


In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(train_labels))

train_labels_one_hot = to_categorical(train_labels, num_classes)
val_labels_one_hot = to_categorical(val_labels, num_classes)
test_labels_one_hot = to_categorical(test_labels, num_classes)

print("One-hot encoded train labels shape:", train_labels_one_hot.shape)
print("One-hot encoded validation labels shape:", val_labels_one_hot.shape)
print("One-hot encoded test labels shape:", test_labels_one_hot.shape)


One-hot encoded train labels shape: (71, 4)
One-hot encoded validation labels shape: (15, 4)
One-hot encoded test labels shape: (16, 4)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(350, 99)),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=5, activation='elu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),
    Conv1D(64, kernel_size=5, activation='elu', kernel_regularizer=l2(0.02)),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    padded_train_data, train_labels_one_hot,
    validation_data=(padded_val_data, val_labels_one_hot),
    epochs=50,
    batch_size=16
)

test_loss, test_accuracy = model.evaluate(padded_test_data, test_labels_one_hot)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
model.save('trained_model11.h5')
print("Model saved as 'trained_model11.h5'")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.3870 - loss: 3.1605 - val_accuracy: 0.2667 - val_loss: 3.0883
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.2717 - loss: 3.0786 - val_accuracy: 0.2667 - val_loss: 3.0218
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.2791 - loss: 3.1798 - val_accuracy: 0.6667 - val_loss: 2.8076
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.4653 - loss: 2.7955 - val_accuracy: 0.4000 - val_loss: 2.7888
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.4850 - loss: 2.7193 - val_accuracy: 0.5333 - val_loss: 2.6173
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.5447 - loss: 2.4859 - val_accuracy: 0.6000 - val_loss: 2.4147
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.7502 - loss: 2.1811 - val_accuracy: 0.7333 - val_loss: 2.3209
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.5699 - loss: 2.2635 - val_accuracy: 0.7333 - val_loss: 2.1799
Epoch 9/50

Test Loss: 1.1780929565429688, Test Accuracy: 0.875
Model saved as 'trained_model11.h5'
